### Các bước chuẩn bị

In [2]:
import os
import time
import re
import random
import requests
import random
import json
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


def read_config(file_path):
    config = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_config(r'C:\working\job_rcm\job_rcm_code\config.txt')

PROXY_HOST = proxy_config.get('PROXY_HOST', '')
PROXY_PORT = proxy_config.get('PROXY_PORT', '')
PROXY_USER = proxy_config.get('PROXY_USER', '')
PROXY_PASS = proxy_config.get('PROXY_PASS', '')

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Manifest for proxy
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

# Background script for proxy
background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
# options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument("user-data-dir=C:\\Users\\trand\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 7")

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")

# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)

cookie_file = r'C:\working\job_rcm\job_rcm_code\job_scraping\linkedin\linkedin_cookies.json'

# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

# Load cookies nếu có
try:
    with open(cookie_file, "r") as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    # Refresh để xem có đăng nhập thành công không
    driver.refresh()
    time.sleep(3)

    if "login" not in driver.current_url:
        print("Đăng nhập bằng cookie thành công!")
    else:
        print("Cookie hết hạn, đăng nhập lại...")
        raise Exception("Cookie Expired")
except Exception as e:
    print("Không thể đăng nhập bằng cookie:", str(e))
    
    # Đăng nhập bằng tài khoản & mật khẩu
    time.sleep(4)
    email = driver.find_element(By.NAME, "session_key")
    password = driver.find_element(By.NAME, "session_password")
    email.send_keys("luantran121204@gmail.com")
    time.sleep(1)
    password.send_keys("tranduyluan11062003")
    time.sleep(2)

    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    # Chờ trang load hoàn tất
    time.sleep(5)

    # Kiểm tra nếu đăng nhập thành công, lưu cookie mới
    if "login" not in driver.current_url:
        print("Đăng nhập thành công! Lưu cookie mới...")
        
        # Lưu cookie vào file JSON
        cookies = driver.get_cookies()
        with open(cookie_file, "w") as file:
            json.dump(cookies, file, indent=4)
        
        print("Cookies đã được lưu vào", cookie_file)
    else:
        print("Đăng nhập thất bại! Vui lòng kiểm tra lại thông tin tài khoản.")

# Login 
actions = ActionChains(driver)

time.sleep(3)
button = driver.find_elements(By.CSS_SELECTOR,"li.global-nav__primary-item")
job_btn = button[2]
actions = ActionChains(driver)
actions.move_to_element(job_btn).perform()
time.sleep(1)
actions.click(job_btn).perform()


time.sleep(7)

search_box = driver.find_element(By.CSS_SELECTOR,"input.jobs-search-box__text-input.jobs-search-box__keyboard-text-input.jobs-search-global-typeahead__input")

actions.move_to_element(search_box).perform()
time.sleep(1)

search_box.send_keys("data")
time.sleep(1)
actions.send_keys(Keys.ENTER).perform()

C:\Users\trand\AppData\Local\Temp\ipykernel_1432\1696082685.py:124: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


Đăng nhập bằng cookie thành công!


In [ ]:
message_btn = driver.find_element(By.CSS_SELECTOR,"div.msg-overlay-bubble-header__details.flex-row.align-items-center.ml1")
actions.click(message_btn).perform()

In [ ]:
# Lấy danh sách cookie từ trình duyệt
cookies = driver.get_cookies()

# In cookie ra màn hình
print("Cookies:")
for cookie in cookies:
    print(cookie)

# Lưu cookie vào file JSON để sử dụng sau này
with open("linkedin_cookies.json", "w") as file:
    json.dump(cookies, file, indent=4)

print("Cookies đã được lưu vào file facebook_cookies.json")

### Lấy thành phần

In [ ]:
# giai đoạn này chỉ cần thực hiện được các tác vụ và lấy được các thành phần

In [ ]:
# lấy được href của các job 
'''
Muốn lấy được href các trang thì cần phải phải 
	click được qua từng trang tiếp theo 
	tìm và thu được href từ từng trang
    
'''

#### lấy href từng job

In [17]:
job_elements = driver.find_elements(By.CSS_SELECTOR,"li.ember-view.jobs-search-results__list-item.occludable-update.p0.relative.scaffold-layout__list-item")

for job in job_elements:
    actions.move_to_element(job).perform()
    job_href = job.find_element(By.CSS_SELECTOR,"div.full-width.artdeco-entity-lockup__title.ember-view a").get_attribute("href")
    print(job_href)

https://www.linkedin.com/jobs/view/4037932407/?eBP=CwEAAAGSf96NPvpkVyeBxCnygXdFC1ztG_mYAYA2SF28ia8jXfyN4VVYBRRI8UGGlhcHlxDGrKXxwIkBtf_A76Lg63biFGCugTyX38Co46J7EvNgvov4xnOW0MOWCihWHselkNaN8azI98D8GZ3AQ-FaZFwy_kanH8QAlj2IgwFDi3VlVng4aLkcOD359gqbVwR42sxRi6yYfZDf_Z08vL6I-Z9r1FwfuH8n5Pt3q3ta7gEInQx6UjuqqBw8RANp5KQ3Qiv4bZNPBwaaZ9-j0k_8oKWfBF2aBddJTJi3J8sYS-qihRjyoEBf6Vv-pKjNbg_hLsA7231xiKGeJrq_BJ6S8PnbxhyuFbXw23wf16x2KlFKxq7JyoGdv3j1dN7T5JBCrZDBiUNzTdVQ7Z94HTgPJG9EJBFcamhCPQ_JRZnwTjiCCsf3YiplvTDAiIKR1CU4iTxdDRXcmWyLQVwDnaHXtYF4SSjAb-aM87dISTSjeqzvWWISM1ah3H8&refId=G%2FFuhy%2FadKbINwkkRRY23Q%3D%3D&trackingId=HGkLsoPkt1HpUmECvcwv1g%3D%3D&trk=flagship3_search_srp_jobs
https://www.linkedin.com/jobs/view/4036727659/?eBP=CwEAAAGSf96NPhdM4EhmT2XnDaX4v13TJt1EZT9dxqmrqXXm9plE_Dy31XdTUBXjS7uuk1rjGZynczhk-dLmB4zbBSN5cT6nlc6HtWsb_DNO5Ec49O_drFvRyoV6SxVvFkDt4Gitkxn5-iRxB-G7YEyJsAg08uAw-V2GihkYjk6K538yxb9019WDBDdyOAjeSob15WWYwcYBdgk0PjrLskQuzPi8NO-9Aqfc_0l6zjvxFXXPHtQXx7eu2EV8sMnpA9trZ3RnuxdrvjInsPjIFpzTT

#### lấy detail từng post

In [ ]:
name = ''
company = ''
city/region = ''
level = ''
HR_href = ''
JD/JRe = ''


In [33]:
detail_box_element = driver.find_element(By.CSS_SELECTOR,"div.scaffold-layout__detail.overflow-x-hidden.jobs-search__job-details")

name = detail_box_element.find_element(By.CSS_SELECTOR,'h1.t-24.t-bold.inline').text
print(name)

company_href = detail_box_element.find_element(By.CSS_SELECTOR,'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
print(company_href)

city_region = detail_box_element.find_element(By.CSS_SELECTOR,'span.tvm__text.tvm__text--low-emphasis').text
print(city_region)

level = detail_box_element.find_elements(By.CSS_SELECTOR,'span.job-details-jobs-unified-top-card__job-insight-view-model-secondary')[1].text
print(level)

HR_href = detail_box_element.find_element(By.CSS_SELECTOR,'div.display-flex.align-items-center.job-details-module__content a').get_attribute("href")
# print(HR_href)

JD_box = detail_box_element.find_element(By.CSS_SELECTOR,'div.jobs-box--fadein.jobs-box--full-width.jobs-box--with-cta-large.jobs-description.jobs-description--reformatted.job-details-module')

JD_JRe_elements = JD_box.find_elements(By.CSS_SELECTOR,'div.job-details-module__content span')
JD_JRe = ''
for element in JD_JRe_elements: 
    JD_JRe += element.text + "\n"
print(JD_JRe)
    

Tester - Data Technology Center - Data Division
https://www.linkedin.com/company/mbbank/life
Hanoi Capital Region
Mid-Senior level
Job Description:
Job Description:
Learn and research documents, systems, requirements and mapping to understand content needs to be tested 
Test planning, designing and building test scenarios, writing test cases and performing system and integration tests 
Performing tests, logging errors, determining severity, prioritizing each error and tracking bug-fix progress 
Statistical reporting and analysis of error causes 
Coordinate UAT operations on completed test requirements. User training and support
Report to the team leader on any problems that need to be resolved during the testing process such as: resources, testing environment, level of complexity...
Learn and research documents, systems, requirements and mapping to understand content needs to be tested 
 
Test planning, designing and building test scenarios, writing test cases and performing system and

# Vòng lặp chuyển trang

In [10]:
# Hàm duyệt qua từng trang
def navigate_pages(n):
    for page in range(2, n + 1):
					# Xác định CSS Selector cho mỗi trang
					css_selector = f'li[data-test-pagination-page-btn="{page}"] button'
					page_button = driver.find_element(By.CSS_SELECTOR, css_selector)
					actions.move_to_element(page_button).perform()
					time.sleep(1)
					actions.click(page_button).perform()
					print(f"Đang ở trang {page}")
					# Thêm thời gian chờ để trang load
					time.sleep(10) 


In [ ]:
navigate_pages(17)

# hoàn chỉnh sơ bộ

In [15]:
# Hàm duyệt qua từng trang
def list_page(n):
    for page in range(2, n + 1):
        
        get_job_links()

        # Xác định CSS Selector cho mỗi trang
        # css_selector = f'li[data-test-pagination-page-btn="{page}"] button'
        css_selector = f'li.jobs-search-pagination__indicator button[aria-label="Page {page}"]'

        try:
            page_button = driver.find_element(By.CSS_SELECTOR, css_selector)
            actions.move_to_element(page_button).perform()
            time.sleep(1)
            actions.click(page_button).perform()
            print(f"Đang ở trang {page}")
            # Thêm thời gian chờ để trang load
            time.sleep(10)
        except:
            print(f"Không thể tìm thấy nút trang {page}, dừng duyệt trang.")
            break

# Hàm lấy link các job trong trang hiện tại
def get_job_links():
    job_elements = driver.find_elements(By.CSS_SELECTOR, "li.ember-view.occludable-update.p0.relative.scaffold-layout__list-item")
    
    for job in job_elements:
        actions.move_to_element(job).perform()
        time.sleep(1)

        try:
            # Tìm phần tử chứa link
            job_href = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link").get_attribute("href")
            print(job_href)
        except Exception as e:
            print(f"Lỗi khi lấy link job: {e}")



In [ ]:
list_page(30)

# Hoàn chỉnh

In [2]:
import os
import time
import re
import random
import requests
import random
import json
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


### Các bước chuẩn bị
def read_config(file_path):
    config = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_config(r'C:\working\job_rcm\job_rcm_code\config.txt')

PROXY_HOST = proxy_config.get('PROXY_HOST', '')
PROXY_PORT = proxy_config.get('PROXY_PORT', '')
PROXY_USER = proxy_config.get('PROXY_USER', '')
PROXY_PASS = proxy_config.get('PROXY_PASS', '')

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Manifest for proxy
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

# Background script for proxy
background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
# options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument("user-data-dir=C:\\Users\\trand\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 7")

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")

# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)

cookie_file = r'C:\working\job_rcm\job_rcm_code\job_scraping\linkedin\linkedin_cookies.json'

# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

# Load cookies nếu có
try:
    with open(cookie_file, "r") as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    # Refresh để xem có đăng nhập thành công không
    driver.refresh()
    time.sleep(3)

    if "login" not in driver.current_url:
        print("Đăng nhập bằng cookie thành công!")
    else:
        print("Cookie hết hạn, đăng nhập lại...")
        raise Exception("Cookie Expired")
except Exception as e:
    print("Không thể đăng nhập bằng cookie:", str(e))
    
    # Đăng nhập bằng tài khoản & mật khẩu
    time.sleep(4)
    email = driver.find_element(By.NAME, "session_key")
    password = driver.find_element(By.NAME, "session_password")
    email.send_keys("luantran121204@gmail.com")
    time.sleep(1)
    password.send_keys("tranduyluan11062003")
    time.sleep(2)

    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    # Chờ trang load hoàn tất
    time.sleep(5)

    # Kiểm tra nếu đăng nhập thành công, lưu cookie mới
    if "login" not in driver.current_url:
        print("Đăng nhập thành công! Lưu cookie mới...")
        
        # Lưu cookie vào file JSON
        cookies = driver.get_cookies()
        with open(cookie_file, "w") as file:
            json.dump(cookies, file, indent=4)
        
        print("Cookies đã được lưu vào", cookie_file)
    else:
        print("Đăng nhập thất bại! Vui lòng kiểm tra lại thông tin tài khoản.")

# # Login 
# actions = ActionChains(driver)

# time.sleep(3)
# button = driver.find_elements(By.CSS_SELECTOR,"li.global-nav__primary-item")
# job_btn = button[2]
# actions = ActionChains(driver)
# actions.move_to_element(job_btn).perform()
# time.sleep(1)
# actions.click(job_btn).perform()
# time.sleep(5)

# job_for_u_btn= driver.find_elements(By.CSS_SELECTOR,"a.uHSoMZelKarpMoXPaKUneRvjGHSMsCSIAOAk.discovery-templates-jobs-home-vertical-list__footer.artdeco-button.artdeco-button--2.artdeco-button--tertiary.artdeco-button--muted.artdeco-button--fluid.pv3.ph2.artdeco-button--icon-right")
# # print(job_for_u_btn[0].get_attribute("href"))


# actions.move_to_element(job_for_u_btn[0]).perform()
# actions.click(job_for_u_btn[0]).perform()


# time.sleep(5)
# search_box = driver.find_element(By.CSS_SELECTOR,"input.jobs-search-box__text-input.jobs-search-box__keyboard-text-input.jobs-search-global-typeahead__input")

# actions.move_to_element(search_box).perform()
# time.sleep(1)

# search_box.send_keys("data")
# time.sleep(1)
# actions.send_keys(Keys.ENTER).perform()

C:\Users\trand\AppData\Local\Temp\ipykernel_9704\2942931815.py:123: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


Đăng nhập bằng cookie thành công!


## LISTVIEW

In [ ]:
keyword_list = set() # đọc từ file csv , đi qua một lần từ từ đầu tiên đến từ cuối cùng, dừng vòng lặp khi hết từ trong list

In [ ]:
#listview_code

for word in keyword_list : 
		search_box = driver.find_element(By.CSS_SELECTOR,"input.jobs-search-box__text-input.jobs-search-box__keyboard-text-input.jobs-search-global-typeahead__input")
		actions.move_to_element(search_box).perform()
		search_box.send_keys(word)
		actions.send_keys(Keys.ENTER).perform()

		# def switch_page :
		#		def get_job_links : 


exit()

In [ ]:
job_elements = driver.find_elements(By.CSS_SELECTOR, "li.ember-view.occludable-update.scaffold-layout__list-item")
print(len(job_elements))

for job in job_elements:
			job_href = job.find_element(By.CSS_SELECTOR, "div.full-width.artdeco-entity-lockup__title.ember-view a").get_attribute("href")
			print(job_href)

In [ ]:
def get_job_links(page):
    job_elements = driver.find_elements(By.CSS_SELECTOR, "li.ember-view.occludable-update.scaffold-layout__list-item")
    print(len(job_elements))
    for job in job_elements:
        actions.move_to_element(job).perform()
        job_href = job.find_element(By.CSS_SELECTOR, "div.full-width.artdeco-entity-lockup__title.ember-view a").get_attribute("href")
        print(job_href)

def page_switch(n):
    for page in range(2, n + 1):
        get_job_links(page)
        css_selector = f'li.jobs-search-pagination__indicator button[aria-label="Page {page}"]'
        page_button = driver.find_element(By.CSS_SELECTOR, css_selector)
        actions.move_to_element(page_button).perform()
        time.sleep(1)
        actions.click(page_button).perform()
        print(f"Đang ở trang {page}")

In [16]:
page_switch(41)	

Đang ở trang 2
Đang ở trang 3
Đang ở trang 4
Đang ở trang 5
Đang ở trang 6
Đang ở trang 7
Đang ở trang 8
Đang ở trang 9
Đang ở trang 10
Đang ở trang 11
Đang ở trang 12
Đang ở trang 13
Đang ở trang 14
Đang ở trang 15
Đang ở trang 16
Đang ở trang 17
Đang ở trang 18
Đang ở trang 19
Đang ở trang 20
Đang ở trang 21


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"li.jobs-search-pagination__indicator button[aria-label="Page 22"]"}
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF78A56C6A5+28789]
	(No symbol) [0x00007FF78A4D5B20]
	(No symbol) [0x00007FF78A368F9A]
	(No symbol) [0x00007FF78A3BF346]
	(No symbol) [0x00007FF78A3BF57C]
	(No symbol) [0x00007FF78A412B17]
	(No symbol) [0x00007FF78A3E736F]
	(No symbol) [0x00007FF78A40F7E3]
	(No symbol) [0x00007FF78A3E7103]
	(No symbol) [0x00007FF78A3AFFC0]
	(No symbol) [0x00007FF78A3B1273]
	GetHandleVerifier [0x00007FF78A8B1AED+3458237]
	GetHandleVerifier [0x00007FF78A8C829C+3550316]
	GetHandleVerifier [0x00007FF78A8BDB9D+3507565]
	GetHandleVerifier [0x00007FF78A632C6A+841274]
	(No symbol) [0x00007FF78A4E09EF]
	(No symbol) [0x00007FF78A4DCB34]
	(No symbol) [0x00007FF78A4DCCD6]
	(No symbol) [0x00007FF78A4CC119]
	BaseThreadInitThunk [0x00007FFA88D6E8D7+23]
	RtlUserTh

In [ ]:
def get_job_links(page):
    job_elements = driver.find_elements(By.CSS_SELECTOR, "li.ember-view.occludable-update.scaffold-layout__list-item")
    
    with open(json_file, 'r+') as f:
        data = json.load(f)
        page_data = data[f'page {page}']
        # Duyệt qua các công việc
        for index, job in enumerate(job_elements, start=1):
            actions.move_to_element(job).perform()  # Cuộn tới phần tử công việc

            try:
                # Lấy link công việc
                job_href = job.find_element(By.CSS_SELECTOR, "div.full-width.artdeco-entity-lockup__title.ember-view a").get_attribute("href")
                print(job_href)
                # Lưu vào JSON
                page_data['jobs'][f'job_{index}'] = {'link_job': job_href}
            except Exception as e:
                # Ghi giá trị 'null' nếu xảy ra lỗi
                print(f"Lỗi khi lấy link công việc: {e}")
                page_data['jobs'][f'job_{index}'] = {'link_job': 'null'}
        
        # Ghi lại dữ liệu vào file JSON
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()  # Xóa các nội dung dư thừa
        
def page_switch(n):
    for page in range(1, n + 1):
        time.sleep(4)
        
        # Lấy HTML trang hiện tại và lưu lại
        html_content = driver.page_source
        save_html_and_update_json(page, html_content)

        # Lấy link các job trên trang và cập nhật file JSON
        get_job_links(page)

        # Chuyển trang
        css_selector = f'li.jobs-search-pagination__indicator button[aria-label="Page {page + 1}"]'
        try:
            page_button = driver.find_element(By.CSS_SELECTOR, css_selector)
            actions.move_to_element(page_button).perform()
            time.sleep(1)
            actions.click(page_button).perform()
            print(f"Đang ở trang {page + 1}")
            # time.sleep(10)
        except:
            print(f"Không thể tìm thấy nút trang {page + 1}, dừng duyệt trang.")
            break

In [6]:
list_info_add = pd.read_csv(r'C:\working\job_rcm\data\linkedin\2025\03\15\Attempt_13_18\list_info_add.csv')
list_info_add.head()

,page,job_index,job_link
0,1,1,https://www.linkedin.com/jobs/view/4178680569/...
1,1,2,https://www.linkedin.com/jobs/view/4179601755/...
2,1,3,https://www.linkedin.com/jobs/view/4113887034/...
3,1,4,https://www.linkedin.com/jobs/view/4174202341/...
4,1,5,https://www.linkedin.com/jobs/view/4178041781/...


In [4]:
### Các bước chuẩn bị
def read_config(file_path):
    config = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_config(r'C:\working\job_rcm\job_rcm_code\config.txt')

PROXY_HOST = proxy_config.get('PROXY_HOST', '')
PROXY_PORT = proxy_config.get('PROXY_PORT', '')
PROXY_USER = proxy_config.get('PROXY_USER', '')
PROXY_PASS = proxy_config.get('PROXY_PASS', '')

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Manifest for proxy
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

# Background script for proxy
background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
# options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument("user-data-dir=C:\\Users\\trand\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 7")

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")

# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)
# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)

C:\Users\trand\AppData\Local\Temp\ipykernel_22572\1828344170.py:102: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


In [23]:
from datetime import datetime

In [ ]:
# #cũ
# def detail_scraping(link):
#     print(f"Đang xử lý link: {link}")
    
# 		#job name
#     job_relative_box_element = driver.find_element(By.CSS_SELECTOR, "div.t-14.artdeco-card")
#     name = job_relative_box_element.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.inline').text
    
# 		#job status
#     try:
#         job_status = job_relative_box_element.find_element(By.CSS_SELECTOR, "span.artdeco-inline-feedback__message").text
#     except:
#         job_status = "available"
        
# 		#company name and company href
#     company_href = job_relative_box_element.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
#     company_name = job_relative_box_element.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name a").text
    
# 		#city region
#     city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR, 'span.tvm__text.tvm__text--low-emphasis')[0].text
    
# 		#scrape date
#     scrape_date = datetime.now().isoformat()
    
# 		#hr id
#     try:
#         hr_box = driver.find_element(By.CSS_SELECTOR, "div.job-details-people-who-can-help__section")
#         hr_id = hr_box.find_element(By.CSS_SELECTOR, "div.display-flex.align-items-center.mt4 a").get_attribute("href")
#     except:
#         hr_id = None
        
# 		#job description
#     jd_see_more = driver.find_element(By.CSS_SELECTOR,'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
#     ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
#     time.sleep(1)
#     jd = driver.find_element(By.CSS_SELECTOR,"div.jobs-description__content.jobs-description-content.jobs-description__content--condensed").text
    
# 		#job data
#     job_data = {
# 				"job": {
# 						"company_name": company_name,
# 						"company_href": company_href,
# 						"job_name": name,
# 						"region": city_region,
# 						"job_status": job_status,
# 						"scrape_date": scrape_date,
# 						"hr_id": hr_id,
# 						"jd": jd
# 				}
# 		}
    
#     print(json.dumps(job_data, indent=4, ensure_ascii=False))

    

In [14]:
def get_job_id(job_link):
    match = re.search(r'view/(\d+)/', job_link)
    if match:
        return match.group(1)  # Lấy số job_id
    return None

In [37]:
def detail_scraping(link):
    
    print(f"Đang xử lý link: {link}")
    time.sleep(2)
    #job name
    print('job_name')
    job_relative_box_element = driver.find_element(By.CSS_SELECTOR, "div.t-14.artdeco-card")
    name = job_relative_box_element.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.inline').text
    time.sleep(2)
    #job status
    print('job_status')
    try:
        job_status = job_relative_box_element.find_element(By.CSS_SELECTOR, "span.artdeco-inline-feedback__message").text
    except:
        job_status = "available"
    time.sleep(2)
    #company name and company href
    print('company_name and company_href')
    company_href = job_relative_box_element.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
    company_name = job_relative_box_element.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name a").text
    time.sleep(2)
    #city region
    city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR, 'span.tvm__text.tvm__text--low-emphasis')[0].text
    time.sleep(2)
		#scrape date
    scrape_date = datetime.now().isoformat()
    
		#hr id
    try:
        hr_box = driver.find_element(By.CSS_SELECTOR, "div.job-details-people-who-can-help__section")
        hr_id = hr_box.find_element(By.CSS_SELECTOR, "div.display-flex.align-items-center.mt4 a").get_attribute("href")
    except:
        hr_id = None
        
		#job description
    jd_see_more = driver.find_element(By.CSS_SELECTOR,'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
    ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
    time.sleep(1)
    jd = driver.find_element(By.CSS_SELECTOR,"div.jobs-description__content.jobs-description-content.jobs-description__content--condensed").text
    
		#job data
    job_data = {
				"job": {
						"company_name": company_name,
						"company_href": company_href,
						"job_name": name,
						"region": city_region,
						"job_status": job_status,
						"scrape_date": scrape_date,
						"hr_id": hr_id,
						"jd": jd
				}
		}
    print(job_data)
    job_id = get_job_id(link)
    detailview_dir = r'C:\working\job_rcm\data\linkedin\2025\03\15\Attempt_13_18\detailview'
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    job_folder = os.path.join(detailview_dir, f"{job_id}_{timestamp}")
    os.makedirs(job_folder, exist_ok=True)

    html_file_path = os.path.join(job_folder, "page_source.html")
    with open(html_file_path, "w", encoding="utf-8") as html_file:
        html_file.write(driver.page_source)

    json_file_path = os.path.join(job_folder, "data.json")
    with open(json_file_path, "w", encoding="utf-8") as json_file:
        json.dump(job_data, json_file, indent=4, ensure_ascii=False)

    print(f" Đã lưu dữ liệu cho {job_id} vào {job_folder}")
    
    print(json.dumps(job_data, indent=4, ensure_ascii=False))


In [27]:
driver.execute_script(f"window.open('{link}', '_blank');")

In [38]:
for link in list_info_add['job_link']:
		# Mở tab mới
		try:	
				driver.execute_script(f"window.open('{link}', '_blank');")
				time.sleep(2)  # Chờ tab mới load (có thể điều chỉnh)
		except:
				print("Không thể mở tab mới!")
				continue

		# Chuyển sang tab mới
		driver.switch_to.window(driver.window_handles[-1])
		print(f"Đang vào: {link}")
		time.sleep(5)  # Chờ trang load (có thể điều chỉnh)
		detail_scraping(link)
		print('đã xử lý xong')

		# Kiểm tra số lượng tab, nếu > 1 thì đóng tab cũ
		if len(driver.window_handles) > 1:
				# time.sleep(10)
				driver.close()  # Đóng tab hiện tại
				driver.switch_to.window(driver.window_handles[0])  # Quay về tab chính
# Đóng trình duyệt khi xong
driver.quit()

Đang vào: https://www.linkedin.com/jobs/view/4178680569/?eBP=CwEAAAGVmHQOVG70QM9WpDi8SFvI7GKoelfRwEmAyPa_2_wupR7jBLwAT-MmCIxhpNv5QaymDa7Veagu-8RLHEQ1YziItiO2VaLZgM7avHsxWfpCuHR5HPNREBAvgz2djvySyZblkd4-an45bFrGEEzbh6YszrDRsU8b4TqUTa3RQT2L6asfs0uRLafYX_PvcLbuZwKfBDPQKyfNjyzi_GEQYdQRXzcm8DYDurhVtgbf_A7I0X2baE0pDij6QBUwxf3CyVw6J_yz5jwTQp3pIoDfsytOYvysPgF6UxSosKLnE69uDBZ4YmsRahdCv1IRgUVX8IHHz_AORc1md6ua2tzI11upju_RIMo64BZBrT1Adcm66j9j8mhTe9dKf6FG8K1la4s9NLnKvb2CqZPdGIKUXHVuFRAQKzj9Wvw6gSbK85znFqc6XO9LX2QZ-oJAeSl7skRxGV6PAWK82hmelySsMm1P3iOZqYmoiPa90Nznbs8_BO63UsJFcp5q0twEgROd&refId=2N3WFeK13OgDDEuKAxuBiQ%3D%3D&trackingId=U8aM%2FbOabvUs4mbyBFBpWw%3D%3D&trk=flagship3_search_srp_jobs
Đang xử lý link: https://www.linkedin.com/jobs/view/4178680569/?eBP=CwEAAAGVmHQOVG70QM9WpDi8SFvI7GKoelfRwEmAyPa_2_wupR7jBLwAT-MmCIxhpNv5QaymDa7Veagu-8RLHEQ1YziItiO2VaLZgM7avHsxWfpCuHR5HPNREBAvgz2djvySyZblkd4-an45bFrGEEzbh6YszrDRsU8b4TqUTa3RQT2L6asfs0uRLafYX_PvcLbuZwKfBDPQKyfNjyzi_GEQYdQRXzcm8DYDurhVtgbf_A7I0X2baE0

KeyboardInterrupt: 

In [17]:
def get_job_id(job_link):
    match = re.search(r'view/(\d+)/', job_link)
    if match:
        return match.group(1)  # Lấy số job_id
    return None

list_info_add['job_id'] = list_info_add['job_link'].apply(get_job_id)
list_info_add.head()

,page,job_index,job_link,job_id
0,1,1,https://www.linkedin.com/jobs/view/4178680569/...,4178680569
1,1,2,https://www.linkedin.com/jobs/view/4181268095/...,4181268095
2,1,3,https://www.linkedin.com/jobs/view/4182576910/...,4182576910
3,1,4,https://www.linkedin.com/jobs/view/4160814135/...,4160814135
4,1,5,https://www.linkedin.com/jobs/view/4123371957/...,4123371957


## DETAILVEW

quy trình cào detailview 
đăng nhập mở tab như bình thường 
tạo một vòng lặp trong đó sẽ có 
link job lấy từ data.json trong thư mục listview
for link in link_list : 
	mở link đó trong một tab mới 
	tạo một thư mục mới theo nguyên tắc
	thực hiện các thao tác cào 
	lưu vào detail view theo thư mục
	đóng tab link đó 


#### Lấy detailview của một link

In [20]:
import os
import time
import re
import random
import requests
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


### Các bước chuẩn bị
# Function to read proxy config from file
def read_proxy_config(file_path):
    config = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_proxy_config(r'C:\working\job_rcm\job_rcm_code\config.txt')
PROXY_HOST = proxy_config['PROXY_HOST']
PROXY_PORT = proxy_config['PROXY_PORT']
PROXY_USER = proxy_config['PROXY_USER']
PROXY_PASS = proxy_config['PROXY_PASS']

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Manifest for proxy
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

# Background script for proxy
background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
	
# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)

# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")
# driver.get("https://www.linkedin.com/")
# Login 
time.sleep(4)
email = driver.find_element(By.NAME, "session_key")
password = driver.find_element(By.NAME, "session_password")
email.send_keys("luantran121204@gmail.com")
time.sleep(1)

password.send_keys("tranduyluan11062003")
time.sleep(2)

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

C:\Users\trand\AppData\Local\Temp\ipykernel_6704\3574717892.py:120: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


In [13]:
link = 'https://www.linkedin.com/jobs/view/4049023814/?eBP=CwEAAAGSjzwxya5a8XFeGbYQ25PbaXal03lR1ba72oRmmM3GSLwhi208cGc5YPRvH14c8fIoUqSZ2NPRe3lFiTncjL1INgQJE-viA6TIIKydmxA_jyDkoAStlOdwWqLdkos0gHDzAXj4g_4uMayD_tbfcFoNIqnfgBu5j-qsvfGrGzMV0rBknXJ9dAn5PQeAqBaBcMbCDOSNdhzfCshPIp4Z6t5u0P6lxf0BXf1qBrQR8pz4yVCY5RiH-IBeThpgXEg0xfEtFXG8CTce-YZva4OY33IThXnwGvH8Dewj4k9xkO2y-S5G8lueC8PX_BXD2QbFcvGDklMXALu7LO22gOheU6oyP8xJIuP-3WlMUUQK2rW8rhCpzfuqvDrz5xrNWZPDx2QwaH6z_mGxJNHzRHdIT-nS9eKUA29o_EDkHdrtdsqI_F8AOxpS0Jxcc2gj9Es1M8szz16Tg6Lt5W80Q7I6tbd2cJLCwiELlaUicJZIq1o&refId=aSb7lJQO9%2B%2BJ%2BIatn4HW9g%3D%3D&trackingId=nAB9r2HnATeMLJP0ZyY4Ow%3D%3D&trk=flagship3_search_srp_jobs'

# Mở tab mới và truy cập link
driver.execute_script(f"window.open('{link}', '_blank');")

In [14]:
tabs = driver.window_handles

# Chuyển sang tab mới nhất
driver.switch_to.window(tabs[-1])

In [ ]:
job_relative_box_element = driver.find_element(By.CSS_SELECTOR,"div.t-14.artdeco-card")
name = job_relative_box_element.find_element(By.CSS_SELECTOR,'h1.t-24.t-bold.inline').text
print(name)
company_href = job_relative_box_element.find_element(By.CSS_SELECTOR,'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
print(company_href)
company_name = job_relative_box_element.find_element(By.CSS_SELECTOR,"div.job-details-jobs-unified-top-card__company-name a").text
print(company_name)
city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR,'span.tvm__text.tvm__text--low-emphasis')[0].text
print(city_region)



In [ ]:
## có thể có nếu job không còn tuyển, nếu không thì job_status sẽ là available
job_status = job_relative_box_element.find_element(By.CSS_SELECTOR,"span.artdeco-inline-feedback__message").text  
print(job_status)


In [ ]:
# hr_box có thể có hoặc không có, nếu không có thể để null ở hr_id 
hr_box = driver.find_element(By.CSS_SELECTOR,"div.job-details-people-who-can-help__section")
hr_id = hr_box.find_element(By.CSS_SELECTOR,"div.display-flex.align-items-center.mt4 a").get_attribute("href")
print(hr_id)

In [ ]:
jd_see_more = driver.find_element(By.CSS_SELECTOR,'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
time.sleep(1)
jd_box = driver.find_element(By.CSS_SELECTOR,"div.jobs-description__content.jobs-description-content.jobs-description__content--condensed")
print(jd_box.text)

In [ ]:
json= {
  "job": {
    "company_name": "",               # Tên công ty
    "company_href": "",               # Link tới trang công ty
    "job_name": "",                   # Tên công việc
    "region": "",                     # Khu vực
    "job_status": "",                 # #Trạng thái công việc: "available" hoặc "unavailable"
    "scrape_date": "",                # Thời gian lấy dữ liệu (định dạng ISO 8601, đi kèm với job_status)
    "hr_id": "",                      # Mã ID của HR liên hệ
    "jd": ""                          # Mô tả công việc (Job Description)
  }
}

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime

# Giả sử driver đã được khởi tạo và link đã được cung cấp
link = 'https://www.linkedin.com/jobs/view/4049023814/?eBP=CwEAAAGSjzwxya5a8XFeGbYQ25PbaXal03lR1ba72oRmmM3GSLwhi208cGc5YPRvH14c8fIoUqSZ2NPRe3lFiTncjL1INgQJE-viA6TIIKydmxA_jyDkoAStlOdwWqLdkos0gHDzAXj4g_4uMayD_tbfcFoNIqnfgBu5j-qsvfGrGzMV0rBknXJ9dAn5PQeAqBaBcMbCDOSNdhzfCshPIp4Z6t5u0P6lxf0BXf1qBrQR8pz4yVCY5RiH-IBeThpgXEg0xfEtFXG8CTce-YZva4OY33IThXnwGvH8Dewj4k9xkO2y-S5G8lueC8PX_BXD2QbFcvGDklMXALu7LO22gOheU6oyP8xJIuP-3WlMUUQK2rW8rhCpzfuqvDrz5xrNWZPDx2QwaH6z_mGxJNHzRHdIT-nS9eKUA29o_EDkHdrtdsqI_F8AOxpS0Jxcc2gj9Es1M8szz16Tg6Lt5W80Q7I6tbd2cJLCwiELlaUicJZIq1o&refId=aSb7lJQO9%2B%2BJ%2BIatn4HW9g%3D%3D&trackingId=nAB9r2HnATeMLJP0ZyY4Ow%3D%3D&trk=flagship3_search_srp_jobs'  

# Mở tab mới và truy cập link
driver.execute_script(f"window.open('{link}', '_blank');")
tabs = driver.window_handles

time.sleep(3)
# Chuyển sang tab mới nhất
driver.switch_to.window(tabs[-1])

time.sleep(1)

# Lấy tên công việc
job_relative_box_element = driver.find_element(By.CSS_SELECTOR, "div.t-14.artdeco-card")
name = job_relative_box_element.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.inline').text

# Lấy job_status (nếu không có element thì sẽ đặt là "available")
try:
    job_status = job_relative_box_element.find_element(By.CSS_SELECTOR, "span.artdeco-inline-feedback__message").text
except:
    job_status = "available"

# Lấy tên công ty và link công ty
company_href = job_relative_box_element.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
company_name = job_relative_box_element.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name a").text

# Lấy thông tin khu vực
city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR, 'span.tvm__text.tvm__text--low-emphasis')[0].text

# Lấy scrape_date theo định dạng ISO 8601
scrape_date = datetime.now().isoformat()

# Lấy hr_id (dùng try-except vì có thể hr_box không tồn tại)
try:
    hr_box = driver.find_element(By.CSS_SELECTOR, "div.job-details-people-who-can-help__section")
    hr_id = hr_box.find_element(By.CSS_SELECTOR, "div.display-flex.align-items-center.mt4 a").get_attribute("href")
except:
    hr_id = None

# Lấy mô tả công việc (job description)
jd_see_more = driver.find_element(By.CSS_SELECTOR,'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
time.sleep(1)
jd = driver.find_element(By.CSS_SELECTOR,"div.jobs-description__content.jobs-description-content.jobs-description__content--condensed").text

# Tạo JSON tổng hợp
job_data = {
    "job": {
        "company_name": company_name,
        "company_href": company_href,
        "job_name": name,
        "region": city_region,
        "job_status": job_status,
        "scrape_date": scrape_date,
        "hr_id": hr_id,
        "jd": jd
    }
}

# In ra dữ liệu JSON
import json
print(json.dumps(job_data, indent=4, ensure_ascii=False))


In [ ]:
import json

listview_path = 'C:\\working\\job_rcm\\data\\linkedin\\listview\\data.json'

with open(listview_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

links = [job_info["link_job"] for page in data.values() for job_info in page["jobs"].values()]

for link in links:
			try:
					
							print(f"Đang xử lý link: {link}")
							# Mở tab mới và truy cập link
							driver.execute_script(f"window.open('{link}', '_blank');")
							tabs = driver.window_handles

							time.sleep(3)
							# Chuyển sang tab mới nhất
							driver.switch_to.window(tabs[-1])

							time.sleep(1)

							# Lấy tên công việc
							job_relative_box_element = driver.find_element(By.CSS_SELECTOR, "div.t-14.artdeco-card")
							name = job_relative_box_element.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.inline').text

							# Lấy job_status (nếu không có element thì sẽ đặt là "available")
							try:
									job_status = job_relative_box_element.find_element(By.CSS_SELECTOR, "span.artdeco-inline-feedback__message").text
							except:
									job_status = "available"

							# Lấy tên công ty và link công ty
							company_href = job_relative_box_element.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
							company_name = job_relative_box_element.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name a").text

							# Lấy thông tin khu vực
							city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR, 'span.tvm__text.tvm__text--low-emphasis')[0].text

							# Lấy scrape_date theo định dạng ISO 8601
							scrape_date = datetime.now().isoformat()

							# Lấy hr_id (dùng try-except vì có thể hr_box không tồn tại)
							try:
									hr_box = driver.find_element(By.CSS_SELECTOR, "div.job-details-people-who-can-help__section")
									hr_id = hr_box.find_element(By.CSS_SELECTOR, "div.display-flex.align-items-center.mt4 a").get_attribute("href")
							except:
									hr_id = None

							# Lấy mô tả công việc (job description)
							jd_see_more = driver.find_element(By.CSS_SELECTOR,'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
							ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
							time.sleep(1)
							jd = driver.find_element(By.CSS_SELECTOR,"div.jobs-description__content.jobs-description-content.jobs-description__content--condensed").text

							# Tạo JSON tổng hợp
							job_data = {
									"job": {
											"company_name": company_name,
											"company_href": company_href,
											"job_name": name,
											"region": city_region,
											"job_status": job_status,
											"scrape_date": scrape_date,
											"hr_id": hr_id,
											"jd": jd
									}
							}

							# In ra dữ liệu JSON
							print(json.dumps(job_data, indent=4, ensure_ascii=False))

							driver.close()
							driver.switch_to.window(tabs[0])

			except Exception as e:
					# In thông báo lỗi nếu link không hoạt động
					print(f"Link không hoạt động: {link}")

In [ ]:
import json
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import os
import time

# Tập hợp các từ khóa ngành nghề
key_words = {'data', 'dev', 'finance', 'hr', 'marketing', 'sale'}

# Duyệt qua từng từ khóa ngành nghề
for key in key_words: 
    # Đường dẫn tới tệp danh sách công việc theo ngành
    listview_path = f'C:\\working\\job_rcm\\data\\linkedin\\{key}\\listview\\data.json'
    print(listview_path)

    # Tải dữ liệu ban đầu từ tệp data.json trong listview
    with open(listview_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Danh sách các liên kết công việc
    # links = [job["link_job"] for job in data.get("jobs", [])]
    links = [job_data["link_job"] for page_data in data.values() for job_data in page_data.get("jobs", {}).values()]

    # Tạo thư mục chi tiết nếu chưa tồn tại
    detail_view_folder = f'C:\\working\\job_rcm\\data\\linkedin\\detailview\\{key}'
    os.makedirs(detail_view_folder, exist_ok=True)
    
    detail_view_file_path = os.path.join(detail_view_folder, 'data.json')

    # Danh sách để lưu dữ liệu các công việc đã cào
    jobs_data = []

    # Duyệt qua từng link job
    for link in links:
					try:
							print(f"Đang xử lý link: {link}")
							driver.execute_script(f"window.open('{link}', '_blank');")
							tabs = driver.window_handles
							time.sleep(3)
							driver.switch_to.window(tabs[-1])
							time.sleep(1)

							# Lấy thông tin công việc
							job_relative_box_element = driver.find_element(By.CSS_SELECTOR, "div.t-14.artdeco-card")
							name = job_relative_box_element.find_element(By.CSS_SELECTOR, 'h1.t-24.t-bold.inline').text

							# Lấy trạng thái công việc (nếu có)
							try:
									job_status = job_relative_box_element.find_element(By.CSS_SELECTOR, "span.artdeco-inline-feedback__message").text
							except:
									job_status = "available"

							# Lấy tên và link công ty
							company_href = job_relative_box_element.find_element(By.CSS_SELECTOR, 'div.job-details-jobs-unified-top-card__company-name a').get_attribute("href")
							company_name = job_relative_box_element.find_element(By.CSS_SELECTOR, "div.job-details-jobs-unified-top-card__company-name a").text

							# Thông tin khu vực
							city_region = job_relative_box_element.find_elements(By.CSS_SELECTOR, 'span.tvm__text.tvm__text--low-emphasis')[0].text

							# Ngày lấy thông tin
							scrape_date = datetime.now().isoformat()

							# ID của HR nếu có
							try:
									hr_box = driver.find_element(By.CSS_SELECTOR, "div.job-details-people-who-can-help__section")
									hr_id = hr_box.find_element(By.CSS_SELECTOR, "div.display-flex.align-items-center.mt4 a").get_attribute("href")
							except:
									hr_id = None

							# Lấy mô tả công việc
							jd_see_more = driver.find_element(By.CSS_SELECTOR, 'button.jobs-description__footer-button.t-14.t-black--light.t-bold.artdeco-card__action.artdeco-button.artdeco-button--icon-right.artdeco-button--3.artdeco-button--fluid.artdeco-button--tertiary.ember-view')
							ActionChains(driver).move_to_element(jd_see_more).click(jd_see_more).perform()
							time.sleep(1)
							jd = driver.find_element(By.CSS_SELECTOR, "div.jobs-description__content.jobs-description-content.jobs-description__content--condensed").text

							# Tạo JSON cho công việc
							job_data = {
									"company_name": company_name,
									"company_href": company_href,
									"job_name": name,
									"region": city_region,
									"job_status": job_status,
									"scrape_date": scrape_date,
									"hr_id": hr_id,
									"jd": jd
							}
							jobs_data.append(job_data)
							
							# Lưu vào tệp JSON sau mỗi vòng lặp
							with open(detail_view_file_path, 'w', encoding='utf-8') as file:
									json.dump({"jobs": jobs_data}, file, ensure_ascii=False, indent=4)
									print('lưu dữ liệu thành công')

							# Đóng tab
							driver.close()
							driver.switch_to.window(tabs[0])

					except Exception as e:
							print(f"Link không hoạt động: {link}")


C:\working\job_rcm\data\linkedin\dev\listview\data.json
Đang xử lý link: https://www.linkedin.com/jobs/view/4056508150/?eBP=CwEAAAGTBzDlaEtJ4iiKAE1ZYAi4mMMAPBJd4HVF_3QPemhVeJWYaW33-akMrZx41QQ4ZKLU-5w0tMl5ULG2cO_yXq2IrkCDw8Sn6O2pcGwELgr1qPLd7fkPf6O3O9b1I2CYSmI1PAHfSo5CWsO9h2SynCwlrYsdmplZmQcRG9jSP5R4jt9QjrOZUE0ahJrbX0oKpSscv1aaxLnsfwI8qQ-LQoADuB0dwA4i5CuA3rCCRCQUGamFkx2fOjqtSxmmC5b8acJUONJ6GCDe5qbRJBXodID_FSsGxZLn8eOfRI0g1C9EI29YYC2U1wNzGek8zbJbCYwEBbMmEdYJltLVUEqXiz8CXDDWBmYg8M7Hpe6kJGGV2H6IuCQKMT1k9y9V-KDN9F-W6Ybm07xaXYwJpd8sKkUZkr2FFkX-6SrrC-wprldEJMfY16U_1ET1O3sURN8dEmZJmjG0OJ0enrNcgEuHR-qTS6ynFnatfVZrkajysve3P3O4RrIO6p5wrs_7H6fCcYE&refId=9CpotAn9BnOZne5LQwj%2FXA%3D%3D&trackingId=nMkg%2FjP5e5VHDImt%2Bt68dg%3D%3D&trk=flagship3_search_srp_jobs
lưu dữ liệu thành công
Đang xử lý link: https://www.linkedin.com/jobs/view/4061037133/?eBP=CwEAAAGTBzDlaNyZFPCnIfBFBrSD-BzIxmx8bjNiLvDBIFi7w0ecfscWjmKDsNGCpLYWXaczd8sUU2KMI5-YCjdA8oiIjkbfbqIZHqobMUseAJD4JA28nMrSbE-B6dGUDmJjz8tMziyazdXlbH-7Be68og

KeyboardInterrupt: 